# Investigating radiometer data

In this exercise, you will analyze data from a number of different radiometers and learn to determine the calibration coefficient of a pyranometer.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [1]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra

## Step 1: Read dataset

The example dataset that we will be investigating is available [here](https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/example_solar_radiation_dataset.csv). The dataset contains a subset of the measured parameters from DTU's solar radiometer station in Lyngby during the period 2025-08-26 to 2025-09-03.

The first step is to load the data into a pandas DataFrame:

In [2]:
url = 'https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/example_solar_radiation_dataset.csv'

data = pd.read_csv(url, index_col=[0], parse_dates=[0])

data.head()

,CHP1_140049_DNI_mV,SMP22_200060_DHI_Wm2,CMP11_128767_GHI_mV,StarSchenk_7773_GHI_mV,solar_zenith,solar_azimuth
Time,,,,,,
2025-08-26 00:00:00+00:00,-0.002,-1.00,-0.014,-0.005,113.059,12.894
2025-08-26 00:01:00+00:00,-0.002,-1.00,-0.013,-0.005,113.027,13.159
2025-08-26 00:02:00+00:00,-0.002,-1.00,-0.013,-0.005,112.995,13.424
2025-08-26 00:03:00+00:00,-0.002,-1.01,-0.013,-0.005,112.962,13.689
2025-08-26 00:04:00+00:00,-0.002,-1.00,-0.013,-0.005,112.929,13.954


## Step 2: Convert analog signal to irradiance

Convert the analog signal of the following measurements to irradiance (W/m$^2$):
- CMP11_128767_GHI_mV
- CHP1_140049_DNI_mV

The calibration values can be found from the calibration sheet on Learn.

In [3]:
# Update the code block

# Note the naming convention (the last part of the name denotes the unit)
data['CHP1_140049_DNI_Wm2'] = 

data['CMP11_128767_GHI_Wm2'] = 

SyntaxError: invalid syntax (3684878651.py, line 4)

In [ ]:
#### ANSWER #####
data['CHP1_140049_DNI_Wm2'] = data['CHP1_140049_DNI_mV'] * 10**-3 / (7.87*10**-6)

data['CMP11_128767_GHI_Wm2'] = data['CMP11_128767_GHI_mV'] * 10**-3 / (8.02*10**-6)


## Step 3: Calculate GHI from measured DHI and DNI
For the DHI, use the provided column called ``SMP22_200060_DHI_Wm2``.

*Hint: The ``solar_zenith`` column is in degrees, whereas the input to ``np.cos`` has to be in radians.*

In [ ]:
# Write your code here

In [ ]:
#### ANSWER ####

data['ghi_calc'] = data['SMP22_200060_DHI_Wm2'] + data['CHP1_140049_DNI_Wm2']*np.cos(np.deg2rad(data['solar_zenith']))

## Step 4: Investigate the thermal offset of the instruments

Thermal offsets are caused by net emission of radiation to the sky hemisphere. The magnitude of the thermal offsets depends on sky conditions, but also largely on instrument design.

What is the average and maximum thermal offset for the CHP1 pyrheliometer and CMP11 pyranometer?

*Hint: thermal offsets are easily detectable at nighttime when there is no radiation.*

In [ ]:
# Write your code here

#### ANSWER ####
nighttime = data['solar_zenith'] > 93  # define a mask for nighttime timestamps

nighttime_irradiance = data.loc[nighttime, ['CHP1_140049_DNI_Wm2', 'CMP11_128767_GHI_Wm2']]

nighttime_irradiance.plot.hist(
    histtype='step', bins=25,
    xlabel='Nighttime rradiance [W/m$^2$]')

nighttime_irradiance.describe()  # general statistics

## Step 5: Determine calibration coefficient for StarSchenk pyranometer

The dataset contains the analog output of an additional pyranometer (``StarSchenk_7773_GHI_mV``). The StarSchenk pyranometer was used by the Danish Meteorological Institute (DMI) from 2001 to 2024.

Since the calibration coefficient of the instrument is unknown, you have to determine this value by comparing it to the other high-quality measurements.

In [ ]:
# Write your code here

#### ANSWER ####

# First, select subset of data to use for calibration
# The example below only uses data for which the solar elevation is greater than 20 degrees (avoids likely shading conditions)
# and also requires GHI to be minimum 300 W/m^2 (lower uncertainty measurements)
calibration_conditions = (data['solar_zenith'] < 70) & (data['CMP11_128767_GHI_Wm2'] > 300)

StarSchenk_sensitivity = data.loc[calibration_conditions, 'StarSchenk_7773_GHI_mV'].sum() / data.loc[calibration_conditions, 'CMP11_128767_GHI_Wm2'].sum() * 1000

print(f"Sensitivity of StarSchenk pyranometer: {StarSchenk_sensitivity:.1f} μV/(W/m^2))")

## Step 6: Investigate the cosine response of the StartSchenk pyranometer

The cosine response of the pyranometer can be determined by plotting the calibration coefficient as a function of the solar zenith angle:

In [ ]:
# Write your code here

#### ANSWER ####
# Calculate irradiance time series based on derived sensitivity
data['StarSchenk_7773_GHI_Wm2'] = data['StarSchenk_7773_GHI_mV'] *10**-3 / (StarSchenk_sensitivity * 10**-6)

data['StarSchenk_bias'] = data['StarSchenk_7773_GHI_Wm2'] / data['CMP11_128767_GHI_Wm2']

data[data['CMP11_128767_GHI_Wm2']>200].plot.scatter(
    x='solar_zenith', y='StarSchenk_bias', ylabel='Bias [%]', s=1, alpha=0.5)

# Notice a clear sloping trend in the bias (this is due to the instrument having a poor cosine response)